# SVMRouter - Inference

This notebook demonstrates how to use a trained **SVMRouter** for inference.

## 1. Environment Setup

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
from llmrouter.models.svmrouter import SVMRouter
from llmrouter.utils import setup_environment, load_model, get_longformer_embedding
import yaml

setup_environment()

## 2. Load Trained Router

In [ ]:
# Configuration
CONFIG_PATH = "configs/model_config_train/svmrouter.yaml"

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

config['model_path']['load_model_path'] = config['model_path']['save_model_path']

# Create inference config
INFERENCE_CONFIG_PATH = "configs/model_config_test/svmrouter_inference.yaml"
os.makedirs(os.path.dirname(INFERENCE_CONFIG_PATH), exist_ok=True)

with open(INFERENCE_CONFIG_PATH, 'w') as f:
    yaml.dump(config, f)

# Initialize router
router = SVMRouter(yaml_path=INFERENCE_CONFIG_PATH)
print(f"Router loaded with {len(router.llm_data)} LLM candidates")

## 3. Single Query Routing

In [ ]:
# Example queries
EXAMPLE_QUERIES = [
    {"query": "What is the capital of France?"},
    {"query": "Solve the equation: 2x + 5 = 15"},
    {"query": "Write a Python function to check if a number is prime."},
    {"query": "Explain quantum computing in simple terms."},
]

print("Routing Results:")
print("=" * 60)

for i, query in enumerate(EXAMPLE_QUERIES, 1):
    result = router.route_single(query)
    print(f"{i}. {query['query'][:50]}...")
    print(f"   Routed to: {result['model_name']}")

## 4. Routing Probabilities

In [ ]:
import matplotlib.pyplot as plt

# Load trained model
model_path = os.path.join(PROJECT_ROOT, config['model_path']['load_model_path'])
svm_model = load_model(model_path)

def get_routing_probabilities(query_text):
    embedding = get_longformer_embedding(query_text).numpy().reshape(1, -1)
    proba = svm_model.predict_proba(embedding)[0]
    return dict(sorted(zip(svm_model.classes_, proba), key=lambda x: x[1], reverse=True))

# Visualize for first query
query = EXAMPLE_QUERIES[0]['query']
probs = get_routing_probabilities(query)

plt.figure(figsize=(10, 5))
plt.barh(list(probs.keys()), list(probs.values()))
plt.xlabel('Probability')
plt.title(f'Routing Probabilities for: "{query[:40]}..."')
plt.tight_layout()
plt.show()

## 5. Batch Routing

## 6. File-Based Inference

Load queries from a file and save results.

In [ ]:
import json

# Load queries from a JSONL file
def load_queries_from_file(file_path):
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return queries

# Save results to a JSONL file
def save_results_to_file(results, output_path):
    """Save routing results to a JSONL file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
    print(f"Results saved to: {output_path}")

# Example: Load from default query file
QUERY_FILE = "data/example_data/query_data/default_query_test.jsonl"
OUTPUT_FILE = "outputs/svmrouter_results.jsonl"

if os.path.exists(QUERY_FILE):
    # Load queries
    file_queries = load_queries_from_file(QUERY_FILE)
    print(f"Loaded {len(file_queries)} queries from: {QUERY_FILE}")
    
    # Route queries (using route_batch for efficiency)
    file_results = router.route_batch(batch=file_queries[:10])
    print(f"Routed {len(file_results)} queries")
    
    # Save results
    save_results_to_file(file_results, OUTPUT_FILE)
    
    # Show sample results
    print(f"\nSample results:")
    for i, result in enumerate(file_results[:3], 1):
        print(f"  {i}. {result.get('query', '')[:40]}... -> {result['model_name']}")
else:
    print(f"Query file not found: {QUERY_FILE}")
    print("Create a JSONL file with format: {\"query\": \"Your question\"}")

In [ ]:
from collections import Counter

# Route multiple queries
results = [router.route_single(q) for q in EXAMPLE_QUERIES]

# Show distribution
model_counts = Counter(r['model_name'] for r in results)
print("Routing Distribution:")
for model, count in model_counts.most_common():
    print(f"  {model}: {count}")

## Summary

This notebook demonstrated:
1. Loading a trained SVMRouter
2. Single and batch query routing
3. Visualizing routing probabilities

SVMRouter is effective for:
- High-dimensional embedding spaces
- Clear separation between LLM performance profiles